![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

### Uncorrelated Assets

Finding uncorrelated assets allows you to find a portfolio that will, theoretically, be more diversified and resilient to extreme market events. When combined with other indicators and data sources, this can be an important component in building an algorithm that limits drawdown and remains profitable in choppy markets.

The first step is to execute the cell containing our correlation function and then grab the historical data for the securities and use this to fetch returns data.

In [1]:
// QuantBook C# Research Environment
// For more information see https://www.quantconnect.com/docs/research/overview
#load "../QuantConnect.csx"
using MathNet.Numerics.Statistics;

var qb = new QuantBook();

var tickers = new List<string> {"SQQQ", "TQQQ", "TVIX", "VIXY", "SPLV",
           "SVXY", "UVXY", "EEMV", "EFAV", "USMV"};

foreach(var ticker in tickers){
    qb.AddEquity(ticker, Resolution.Minute);
}

// Fetch history
var history = qb.History(qb.Securities.Keys, 150, Resolution.Hour);

In [2]:
// Calculates the % returns of our symbols over the historical data period
public Dictionary<string, List<Double>> GetReturns(IEnumerable<Slice> history){
    var returns = new Dictionary<string, List<Double>>();
    var last = new Dictionary<string, Double>();
    foreach(var slice in history){
        foreach(var symbol in slice.Bars.Keys){
            if(!returns.ContainsKey(symbol)){
                returns.Add(symbol, new List<Double>());
                last.Add(symbol, (Double)slice.Bars[symbol].Close);
            }
            var change = (Double) ((Double)slice.Bars[symbol].Close - last[symbol])/last[symbol];
            returns[symbol].Add(change);
        }    
    }
    return returns;
}

// Calculates the net absolute correlation between a given security and the remaining securities
public Dictionary<string, Double> GetCorrelations(Dictionary<string, List<Double>> returns){
    
    var correlations = new Dictionary<string, Double>();
    
    foreach(var symbol in returns.Keys){
        if(!correlations.ContainsKey(symbol)){
            correlations.Add(symbol, 0);
        }
        
        foreach(var symbol2 in returns.Keys){
            if(symbol == symbol2) {
                continue;
            }
            var corr = Correlation.Pearson(returns[symbol], returns[symbol2]);
            correlations[symbol] += Math.Abs(corr);
        }        
       
    }    
    
    return correlations;
}


Then, we calculate the correlation of the returns, which gives us a correlation matrix. In the GetUncorrelatedAssets function, we figure out which symbols have the lowest overall correlation with the rest of the symbols as a whole -- we want to find the five assets with the lowest average absolute correlation so that we can trade them without fearing that any pair are too highly correlated.

In [3]:
// Get hourly returns
var returns = GetReturns(history);

// Get correlations
var corr = GetCorrelations(returns);
    
// Get 5 assets with least overall correlation
var uncorrelatedAssets = corr.OrderBy(x => x.Value).Take(5);      
  

foreach(var kvp in uncorrelatedAssets){
    Console.WriteLine(kvp.Key + " , " + kvp.Value);
}